#### Kết nối với Qdrant

In [2]:

from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://8953fb93-cd45-4703-96f3-2d7de40931da.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    api_key="2nBQ8f7gIbn-9gbuJ0k6CNRF06LVIpwuvZiMghDNC3de_kIPaGwRuw",
)

#### Import thư viện

In [3]:
import cv2
import torch
import numpy as np
import os
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image, UnidentifiedImageError
from qdrant_client.models import PointStruct
from qdrant_client.models import Distance, VectorParams
from datetime import datetime
import pandas as pd

#### Tạo collection lưu các embedding vector

In [9]:
qdrant_client.create_collection(
    collection_name="Face detection",
    vectors_config=VectorParams(size=512, distance=Distance.ECULID),
)

AttributeError: ECULID

#### Load model facenet

In [4]:
model = InceptionResnetV1(pretrained='vggface2').eval()

mtcnn = MTCNN(image_size=160, margin=0)

#### Tải lên dữ liệu gương mặt cho database

In [5]:
database_dir = './lfw_funneled'

for id, person_name in enumerate(os.listdir(database_dir)):
    person_dir = os.path.join(database_dir, person_name)
    if os.path.isdir(person_dir):
        embeddings = []
        for file_name in os.listdir(person_dir):
            if file_name.endswith(('jpg', 'jpeg', 'png')):
                image_path = os.path.join(person_dir, file_name)
                try:
                    frame = cv2.imread(image_path)
                    faces, _ = mtcnn.detect(frame)
                    if faces is not None:
                        for face in faces:
                            bbox = list(map(int,face.tolist()))
                            # face_img = cv2.rectangle(frame,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255),6)
                            try:
                                face_img = frame[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]
                                face_img = cv2.resize(face_img, (160, 160))
                                face_img = np.transpose(face_img, (2, 0, 1))
                                face_img = torch.tensor(face_img, dtype=torch.float32)
                                face_img = (face_img - 127.5) / 128.0  
                                face_img = face_img.unsqueeze(0) 
                                embedding = model(face_img).detach()
                                embeddings.append(embedding)
                            except Exception as e:
                                print("Error:", e)
                            
                except ValueError as e:
                    print(f"Skipping {file_name} in {person_name}: {e}")
        if embeddings:
            embedding_vector = torch.stack(embeddings).mean(dim=0)
            embedding_vector = embedding_vector*100
            embedding_array = embedding_vector.numpy().tolist()
            operation_info = qdrant_client.upsert(
                collection_name="Face_detection",
                wait=True,
                points=[
                    PointStruct(id=id, vector=embedding_array[0], payload={"name": person_name}),
                ],
            )



Error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



KeyboardInterrupt: 

#### Chạy realtime face detection

In [12]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
data = {
    'Name':[],
    'Time':[]
}
df = pd.DataFrame(data)

cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)

while True:
    ret, frame = cap.read()
    if ret == True:
        faces, _ = mtcnn.detect(frame)
        if faces is not None:
            for face in faces:
                bbox = list(map(int,face.tolist()))
                face_detect = cv2.rectangle(frame,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255),6)
                face_img = frame[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]
                face_img = cv2.resize(face_img, (160, 160))
                face_img = np.transpose(face_img, (2, 0, 1)) 
                face_img = torch.tensor(face_img, dtype=torch.float32)
                face_img = (face_img - 127.5) / 128.0  
                face_img = face_img.unsqueeze(0) 
                embedding = model(face_img).detach() 
                embedding_vector = embedding*100
                embedding_array = embedding_vector.numpy().tolist()

                results = qdrant_client.query_points(
                    collection_name="Face_detection",
                    query=embedding_array[0],
                    limit=1
                )
                first_point = results.points[0]
                score = first_point.score
                if score <=85:
                    name = first_point.payload['name']
                else:
                    name = "Unknown"
                    
                cv2.putText(face_detect, name, (bbox[0], bbox[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                now = datetime.now()
                time = now.strftime("%Y-%m-%d %H:%M:%S")
                new_row = pd.DataFrame({'Name': [name], 'Time' : [time]})
                df = pd.concat([df, new_row], ignore_index=True)
        cv2.imshow("video", frame)

    key = cv2.waitKey(1)
    if key == ord("q"):
        df.to_csv('data.csv')
        break

cv2.destroyAllWindows()